In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, {"output": "How are you?"})

memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message(1,1)
add_message(2,2)
add_message(3,3)
add_message(4,4)

# memory.load_memory_variables({})

add_message(5,5)

memory.load_memory_variables({})

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jae Won, I live in South Korea","Wow that is so cool!")

add_message("South Korea is so Pretty","I wish I could go!!!")

get_history()

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=150, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

def get_history():
    return memory.load_memory_variables({})

add_message("Hi I'm Jae Won, I live in South Korea","Wow that is so cool!")
add_message("South Korea is so Pretty","I wish I could go!!!")
add_message("How far is Korea from Argentina","I don't know how far")
add_message("How far is Korea from Argentina","I don't know how far")
add_message("How far is Korea from Argentina","I don't know how far")
add_message("How far is Korea from Argentina","I don't know how far")
add_message("How far is Korea from Argentina","I don't know how far")

get_history()

In [12]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(llm=llm, return_messages=True)

def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})

add_message("Hi I'm Jae Won, I live in South Korea","Wow that is so cool!")

memory.load_memory_variables({"input":"who is Jae Won"})

{'history': [SystemMessage(content='On Jae Won: Jae Won lives in South Korea.')]}

In [24]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history")

template = """
    You are a helpful AI talking to a human

    {chat_history}
    Human:{question}
    You:
"""

chain = LLMChain(
    llm=llm,
    memory=memory,
    # prompt=PromptTemplate.from_template("{question}"),
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

chain.predict(question="My name is JaeWon")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    
    Human:My name is JaeWon
    You:


> Finished chain.


'Nice to meet you, JaeWon! How can I assist you today?'

In [25]:
chain.predict(question="I live in South Korea")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is JaeWon
AI: Nice to meet you, JaeWon! How can I assist you today?
    Human:I live in South Korea
    You:


> Finished chain.


"That's great to know! How can I assist you with anything related to South Korea or any other topic you have in mind?"

In [26]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human

    Human: My name is JaeWon
AI: Nice to meet you, JaeWon! How can I assist you today?
Human: I live in South Korea
AI: That's great to know! How can I assist you with anything related to South Korea or any other topic you have in mind?
    Human:what is my name?
    You:


> Finished chain.


'Your name is JaeWon.'

In [9]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
# from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, return_messages=True)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])

# chain = LLMChain(
#     llm=llm,
#     memory=memory,
#     prompt=prompt,
#     verbose=True
# )

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result=chain.invoke({"question": question})
    memory.save_context({"input": question},{"output":result.content})
    print(result)


In [10]:
invoke_chain("Hello, My name is Jaewon")

content='Hello Jaewon! How can I assist you today?'


In [11]:
invoke_chain("What is my name?")


content='Your name is Jaewon.'
